In [2]:
import json
import os
import ast
import pandas as pd

In [ ]:

def has_set(d):
    for value in d.values():
        if isinstance(value, set):
            return True
        elif isinstance(value, dict):
            if has_set(value):
                return True
    return False

def convert_sets_to_lists(d):
    new_dict = {}
    for key, value in d.items():
        if isinstance(value, set):
            new_dict[key] = list(value)
        elif isinstance(value, dict):
            new_dict[key] = convert_sets_to_lists(value)
        else:
            new_dict[key] = value
    return new_dict

def check_syntax_errors(dictionary):
    for key, value in dictionary.items():
        if isinstance(value, str):
            try:
                ast.parse(value)
            except SyntaxError as e:
                print(f"SyntaxError in key '{key}': {e}")
        elif isinstance(value, dict):
            check_syntax_errors(value)

def convert_quotes(input_data):
    if isinstance(input_data, dict):
        return {key: str(value).replace("'", '"') if isinstance(value, str) else value for key, value in input_data.items()}

    elif isinstance(input_data, str):
        modified_string = input_data.replace("'", "\"")
        return modified_string
    else:
        return input_data

def delete_one_brace(input_string):
    modified_string = input_string.replace("}}}}", "}}}")
    return modified_string

def save_json(user_dict,filename):
    json_dict = json.dumps(user_dict, indent=2, sort_keys=True, ensure_ascii=False)
    json_filename = f"{filename[:-4]}.json"
    output_folder = './all_processed'           
    # 使用 os.path.join 来拼接文件路径
    output_path = os.path.join(output_folder, json_filename)
    with open(output_path, "w", encoding='utf-8') as f: 
        f.write(json_dict)
    
    print(filename)


In [ ]:

def json_processed(folder_path):
    for filename in os.listdir(folder_path):
        if not filename.endswith('.txt'):
            continue
        
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

In [ ]:
folder_path = './all_extract'
files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
sorted_files = sorted(files)
for filename in sorted_files:
    file_path = os.path.join(folder_path,filename)
    print(filename)
    with open(file_path, 'r') as load_f:
        load_dict = json.load(load_f) 
    
    load_dict = delete_one_brace(load_dict)
    load_dict = convert_quotes(load_dict)
        
    try:
        user_dict = ast.literal_eval(load_dict) 
        
    except SyntaxError:
        input_string = load_dict
        #input_string = input_string[0:]
        #input_string = input_string[:-1]
        #print(input_string)
        user_dict = delete_one_brace(input_string)
        
        user_dict = ast.literal_eval(input_string)

    except TypeError as e:
        print(filename,f"Error: {e}")
        
    if has_set(user_dict):
        user_dict = convert_sets_to_lists(user_dict)   
    else:
        user_dict = user_dict
       
    
    print(filename,type(user_dict))  
    json_dict = json.dumps(user_dict, indent=2, sort_keys=True, ensure_ascii=False)
    json_filename = f"{filename[:-4]}.json"
    output_folder = './all_processed'          

    output_path = os.path.join(output_folder, json_filename)
                

    with open(output_path, "w", encoding='utf-8') as f: 
        f.write(json_dict)
        